# Imports and setting up viz

NB : conda env1 on PC, lam1env on spirit (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_mapping import *

In [ ]:
rivers = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '10m',edgecolor=(0, 0, 0, 0.3), facecolor='none')

# Load files

In [ ]:
date_min_pres = '2010-01-01'
date_max_pres = '2022-12-31'

date_min_fut = '2050-01-01'
date_max_fut = '2062-12-31'

In [ ]:
# lon_min = -13.5
# lon_max = 6.5
# lat_min = 32
# lat_max = 49
lon_min=-10
lon_max=4
lat_min=35.5
lat_max=44

## Format functions

In [ ]:
def format_LMDZ_MO(filename, name, color=None, date_min=None, date_max=None):
    """
    Format the LMDZ-MO output for plotting.
    """
    ds = xr.open_mfdataset(filename)
    ds.attrs['name'] = name
    ds.attrs['plot_color'] = color if color else 'blue'

    rename_dict = {'time_counter':'time'}
    ds = ds.rename(rename_dict)

    ds = ds.coarsen(lat=2, lon=3, boundary='trim').mean()
    
    #restrict
    if lon_min is not None and lon_max is not None and lat_min is not None and lat_max is not None:
        ds = ds.sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))
    else:
        print('No domain restriction applied, missing lon_min, lon_max, lat_min, lat_max')
    if date_min is not None and date_max is not None:
        ds = ds.sel(time=slice(date_min, date_max))
    else:
        print('No date restriction applied, missing date_min, date_max')

    #edit and convert units
    ds['evap'] = ds['evap'] *3600 * 24
    ds['evap'].attrs['units'] = 'mm d⁻¹'
    ds['evap'].attrs['name'] = 'ET'

    ds['precip'] = ds['precip'] *3600 * 24
    ds['precip'].attrs['units'] = 'mm d⁻¹'
    ds['precip'].attrs['name'] = 'Precipitation'

    ds['t2m'] = ds['t2m'] - 273.15
    ds['t2m'].attrs['units'] = '°C'
    ds['t2m'].attrs['name'] = '2-m temperature'

    ds['fluxsens']= -ds['sens']
    ds['fluxsens'].attrs['units'] = 'W m⁻²'
    ds['fluxsens'].attrs['name'] = 'Sensible heat flux'

    ds['fluxlat'] = - ds['flat']
    ds['fluxlat'].attrs['units'] = 'W m⁻²'
    ds['fluxlat'].attrs['name'] = 'Latent heat flux'

    ds['netrad'] = ds['LWdnSFC'] - ds['LWupSFC'] + ds['SWdnSFC'] - ds['SWupSFC']
    ds['netrad'].attrs['name'] = 'Surface net radiation'
    ds['netrad'].attrs['units'] = 'W m⁻²'

    ds['SWnetSFC'] = ds['SWdnSFC'] - ds['SWupSFC']
    ds['SWnetSFC'].attrs['name'] = 'Surface net SW'
    ds['SWnetSFC'].attrs['units'] = 'W m⁻²'

    ds['LWnetSFC'] = ds['LWdnSFC'] - ds['LWupSFC']
    ds['LWnetSFC'].attrs['name'] = 'Surface net LW'
    ds['LWnetSFC'].attrs['units'] = 'W m⁻²'

    ds['P - E'] = ds['precip'] - ds['evap']
    ds['P - E'].attrs['units'] = 'mm d⁻¹'

    #add names and units
    ds['SWdnSFC'].attrs['name'] = 'Surface SW down'
    ds['SWdnSFC'].attrs['units'] = 'W m⁻²'
    ds['LWdnSFC'].attrs['name'] = 'Surface LW down'
    ds['LWdnSFC'].attrs['units'] = 'W m⁻²'

    ds['cldt'] = ds['cldt'] * 100  # Convert to percentage
    ds['cldt'].attrs['name'] = 'Total cloud cover'
    ds['cldt'].attrs['units'] = '%'

    ds['cldl'] = ds['cldl'] * 100  # Convert to percentage
    ds['cldl'].attrs['name'] = 'Low cloud cover'
    ds['cldl'].attrs['units'] = '%'

    ds['cldm'] = ds['cldm'] * 100  # Convert to percentage
    ds['cldm'].attrs['name'] = 'Medium cloud cover'
    ds['cldm'].attrs['units'] = '%'

    ds['cldh'] = ds['cldh'] * 100  # Convert to percentage
    ds['cldh'].attrs['name'] = 'High cloud cover'
    ds['cldh'].attrs['units'] = '%'

    ds['q2m'].attrs['name'] = '2-m specific humidity'
    ds['q2m'].attrs['units'] = 'kg kg⁻¹'
    ds['rh2m'].attrs['name'] = '2-m relative humidity'
    ds['rh2m'].attrs['units'] = '%'
    ds['s_pblh'].attrs['name'] = 'Boundary layer height'
    ds['s_pblh'].attrs['units'] = 'm'
    ds['s_lcl'].attrs['name'] = 'Lifting condensation level'
    ds['s_lcl'].attrs['units'] = 'm'
    
    ds['psol'] = ds['psol'] / 100  # Convert to hPa
    ds['psol'].attrs['name'] = 'Surface pressure'
    ds['psol'].attrs['units'] = 'hPa'

    ds['evap_frac'] = ds['fluxlat'] / (ds['fluxlat'] + ds['fluxsens']) 
    ds['evap_frac'].attrs['name'] = 'Evaporation fraction'
    ds['evap_frac'].attrs['units'] = '-'

    #remove history, description, title, Conventions, timeStamp, uuid, NCO :
    for attr in ['history', 'description', 'title', 'Conventions', 'timeStamp', 'uuid', 'NCO']:
        if attr in ds.attrs:
            del ds.attrs[attr]

    return ds

In [ ]:
def format_ORC_MO(filename, name, color=None, date_min=None, date_max=None):
    """
    Format the ORC output for plotting.
    """
    ds = xr.open_mfdataset(filename)
    ds.attrs['name'] = name

    if color is not None:
        ds.attrs['plot_color'] = color

    rename_dict = {'time_counter':'time'}
    ds = ds.rename(rename_dict)


    ds = ds.coarsen(lat=2, lon=3, boundary='trim').mean()

    #restrict
    if lon_min is not None and lon_max is not None and lat_min is not None and lat_max is not None:
        ds = ds.sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))
    else:
        print('No domain restriction applied, missing lon_min, lon_max, lat_min, lat_max')
    if date_min is not None and date_max is not None:
        ds = ds.sel(time=slice(date_min, date_max))
    else:
        print('No date restriction applied, missing date_min, date_max')


    #edit units and names
    # if 'streamr' in ds:
    #     ds['streamr'].attrs['units'] = 'kg/m²'
    #     ds['streamr'].attrs['long_name'] = 'River reservoir volume'
    # if 'slowr' in ds:
    #     ds['slowr'].attrs['units'] = 'kg/m²'
    #     ds['slowr'].attrs['long_name'] = 'GW reservoir volume'
    # if 'fastr' in ds:
    #     ds['fastr'].attrs['units'] = 'kg/m²'
    #     ds['fastr'].attrs['long_name'] = 'Overland reservoir volume'
    if 'evapot_corr' in ds:
        ds['evapot_corr'] = ds['evapot_corr'] * 3600 * 24
        ds['evapot_corr'].attrs['units'] = 'mm d⁻¹'
        ds['evapot_corr'].attrs['long_name'] = 'PET'
    if 'irrigmap_dyn' in ds and 'Areas' in ds:
        ds['irrig_frac'] = ds['irrigmap_dyn'] / ds['Areas'] *100
        ds['irrig_frac'].attrs['units'] = '%'
        ds['irrig_frac'].attrs['long_name'] = 'Grid cell irrigation fraction'


    # ds['evap_frac'] = ds['fluxlat'] / (ds['fluxlat'] + ds['fluxsens']) 
    # ds['evap_frac'].attrs['name'] = 'Evaporation fraction'
    # ds['evap_frac'].attrs['units'] = '-'
    
    if 'coastalflow' in ds and 'riverflow' in ds:
        ds['coast_and_river'] = ds['coastalflow'] + ds['riverflow']
        ds['coast_and_river'].attrs['units'] = 'm³/s'
        ds['coast_and_river'].attrs['long_name'] = 'Coastal and river flow'
        
    #remove history, description, title, Conventions, timeStamp, uuid, NCO :
    for attr in ['history', 'description', 'title', 'Conventions', 'timeStamp', 'uuid', 'NCO']:
        if attr in ds.attrs:
            del ds.attrs[attr]
        
    return ds

## LMDZ

In [ ]:
present_noirr_dir='../../../JZ_simu_outputs/LAM/LAM_1000_40/CM6'
future_noirr_dir='../../../JZ_simu_outputs/LAM/LAM_1000_40/future_noirr'
future_irr_dir='../../../JZ_simu_outputs/LAM/LAM_1000_40/future_irr'

In [ ]:
filename = '{}/ATM/*histmth.nc'.format(present_noirr_dir)
color= 'orange'
lmdz_pres_noirr = format_LMDZ_MO(filename, 'present, no_irr', color=color, date_min=date_min_pres, date_max=date_max_pres)
lmdz_pres_noirr

In [ ]:
filename = '{}/*histmth.nc'.format(future_noirr_dir)
color = 'red'
lmdz_fut_noirr = format_LMDZ_MO(filename, 'future, no_irr', color=color, date_min=date_min_fut, date_max=date_max_fut)
lmdz_fut_noirr

In [ ]:
filename = '{}/*histmth.nc'.format(future_irr_dir)
color = 'blue'
lmdz_fut_irr = format_LMDZ_MO(filename, 'future, irr', color=color, date_min=date_min_fut, date_max=date_max_fut)
lmdz_fut_irr

## ORC

In [ ]:
filename = '{}/SRF/*sechiba_history.nc'.format(present_noirr_dir)
color = 'orange'
orc_pres_noirr = format_ORC_MO(filename, 'present, no_irr', color=color, date_min=date_min_pres, date_max=date_max_pres)
orc_pres_noirr

In [ ]:
filename = '{}/*sechiba_history.nc'.format(future_noirr_dir)
color = 'red'
orc_fut_noirr = format_ORC_MO(filename, 'future, no_irr', color=color, date_min=date_min_fut, date_max=date_max_fut)
orc_fut_noirr

In [ ]:
filename = '{}/*sechiba_history.nc'.format(future_irr_dir)
color = 'blue'
orc_fut_irr = format_ORC_MO(filename, 'future, irr', color=color, date_min=date_min_fut, date_max=date_max_fut)
orc_fut_irr

## Masks and interp

In [ ]:
#define masks
con_mask_pres=lmdz_pres_noirr['fract_ter']>0.95
ip_mask_pres=polygon_to_mask(lmdz_pres_noirr, iberian_peninsula)

con_mask_fut=lmdz_fut_irr['fract_ter']>0.95
ip_mask_fut=polygon_to_mask(lmdz_fut_irr, iberian_peninsula)

#apply masks
ip_lmdz_pres_noirr = lmdz_pres_noirr.where(ip_mask_pres, drop=False).where(con_mask_pres)
ip_lmdz_fut_noirr = lmdz_fut_noirr.where(ip_mask_fut, drop=False).where(con_mask_fut)
ip_lmdz_fut_irr = lmdz_fut_irr.where(ip_mask_fut, drop=False).where(con_mask_fut)

ip_orc_pres_noirr = orc_pres_noirr.where(ip_mask_pres, drop=False).where(con_mask_pres)
ip_orc_fut_noirr = orc_fut_noirr.where(ip_mask_fut, drop=False).where(con_mask_fut)
ip_orc_fut_irr = orc_fut_irr.where(ip_mask_fut, drop=False).where(con_mask_fut)


In [ ]:
#mean DS
ip_lmdz_pres_noirr_mean = mean_dataset(ip_lmdz_pres_noirr)
ip_lmdz_fut_noirr_mean = mean_dataset(ip_lmdz_fut_noirr)
ip_lmdz_fut_irr_mean = mean_dataset(ip_lmdz_fut_irr)    

# Tests

In [ ]:
stop

## Maps

In [ ]:
#Display map of var for ds
var='netrad'
ds=ip_lmdz_pres_noirr
color_map=reds
min_value=0
max_value=110
# min_value, max_value = None, None

map_ave(ds, var, cmap=color_map, vmin=min_value, vmax=max_value, hex=False)

In [ ]:
#Diff map
var='rh2m'
# ds1=ip_lmdz_fut_noirr_mean
# ds2=ip_lmdz_pres_noirr_mean

ds1=ip_lmdz_fut_irr
ds2=ip_lmdz_pres_noirr

max_value= 7
min_value=-7
# max_value, min_value = None, None

color_map=emb_neutral
# color_map=emb

# title='Evapotranspiration bias (mm d⁻¹, {} - {})'.format( ds1.attrs['name'], ds2.attrs['name'])
title=None

map_diff_ave(ds1, ds2, var, 
             vmin=min_value, vmax=max_value, 
             title=title, cmap=color_map,
            #  sig=True
             )

# diff=ds1[var]-ds2[var]
# title='{} bias, {} vs {} ({})'.format( var, ds1.attrs['name'], ds2.attrs['name'], ds1[var].attrs['units'])
# map_seasons(diff, cmap=color_map,  vmin=min_value, vmax=max_value, title=title, hex=False)

In [ ]:
#Relative diff map
var='cldm'
ds1=ip_lmdz_fut_noirr_mean
ds2=ip_lmdz_pres_noirr_mean

# ds1=ip_lmdz_fut_irr
# ds2=ip_lmdz_fut_noirr

max_value=25
min_value=-max_value
min_value, max_value = None, None

map_rel_diff_ave(ds1, ds2, var, vmin=min_value, vmax=max_value)

In [ ]:
#map for 4 seasons
var='precip'
ds1=lmdz_fut_noirr
ds2=lmdz_fut_irr
max_value=2
min_value=None
# min_value=-max_value

diff=ds1[var]-ds2[var]
title='{} bias, {} vs {} ({})'.format( var, ds1.attrs['name'], ds2.attrs['name'], ds1[var].attrs['units'])
plotvar=diff

# plotvar=ds1[var]
# title='P-E (mm d⁻¹, {})'.format(ds1.attrs['name'])

map_seasons(plotvar, cmap=emb, vmax=max_value, title=title, hex=False)

## Time series

In [ ]:
var='cape'

# ds1 = ip_lmdz_fut_noirr
ds2 = ip_lmdz_fut_noirr
ds3 = ip_lmdz_pres_noirr
ds1 = ip_lmdz_fut_irr

ds_list=[ds1, ds2, ds3]

year_min, year_max = 2048, 2062
year_min, year_max = 2010, 2022
year_min, year_max = 2010, 2062

time_series_ave(ds_list, var,
                title='{} {}'.format(var, ds1[var].attrs['units']),
                ds_colors=True,
                year_min=year_min, year_max=year_max)
seasonal_cycle_ave(ds_list, var, 
                   title='Seasonal cycle of {} ({})'.format(var, ds1[var].attrs['units']),
                   ds_colors=True,
                   year_min=year_min, year_max=year_max)

# Figures

## Present vs Future

### Annual diff maps

In [ ]:
savefig=True
vars=['precip', 'evap', 't2m', 'fluxsens', 'q2m', 's_pblh', 's_lcl', 'rh2m', 'psol']
# vars=['precip']
vmaxs=[2, 0.3, 3, 15, 1e-3, 50, 200, 7, 1.80]
cmaps=[emb_neutral, emb_neutral, emb, emb, emb_neutral, emb, emb, emb_neutral, emb]

ds1= ip_lmdz_fut_noirr
ds2= ip_lmdz_pres_noirr

ds1_mean = ds1.mean(dim='time')
ds2_mean = ds2.mean(dim='time')

for var in vars:
    title='off'
    ylabel='{} diff. ({})'.format(ds1[var].attrs['name'], ds1[var].attrs['units'])
    map_diff_ave(ds1_mean, ds2_mean, var, 
                 vmin=-vmaxs[vars.index(var)], vmax=vmaxs[vars.index(var)],
                title=title, 
                clabel=ylabel,
                cmap=cmaps[vars.index(var)],
                )
    if savefig:
        plt.savefig('figures/chap4/future/diffmap_{}_presfut.png'.format(var), bbox_inches='tight', dpi=300)

In [ ]:
savefig=True
vars=['SWdnSFC', 'LWdnSFC', 'netrad', 'cldt', 'cldl', 'cldm', 'cldh']
# vars=['precip']
vmaxs=[15, 15, 10, 10, 10, 5, 1]
cmap=emb

ds1= ip_lmdz_fut_noirr
ds2= ip_lmdz_pres_noirr

ds1_mean = ds1.mean(dim='time')
ds2_mean = ds2.mean(dim='time')

for var in vars:
    title='off'
    ylabel='{} diff. ({})'.format(ds1[var].attrs['name'], ds1[var].attrs['units'])
    map_diff_ave(ds1_mean, ds2_mean, var, 
                 vmin=-vmaxs[vars.index(var)], vmax=vmaxs[vars.index(var)],
                title=title, 
                clabel=ylabel,
                cmap=cmap
                )
    if savefig:
        plt.savefig('figures/chap4/future/diffmap_{}_presfut.png'.format(var), bbox_inches='tight', dpi=300)

### Summer diff maps

In [ ]:
savefig=True
vars=['precip', 'evap', 't2m', 'fluxsens', 'q2m', 's_pblh', 's_lcl', 'rh2m']
# vars=['precip']
vmaxs=[2, 1, 4, 30, 1e-3, 150, 450, 10]
cmaps=[emb_neutral, emb_neutral, emb, emb, emb_neutral, emb, emb, emb_neutral]

ds1= ip_lmdz_fut_noirr
ds2= ip_lmdz_pres_noirr

months=[6,7,8]  # JJA
ds1 = ds1.sel(time=ds1['time.month'].isin(months))
ds2 = ds2.sel(time=ds2['time.month'].isin(months))
ds1_mean = ds1.mean(dim='time')
ds2_mean = ds2.mean(dim='time')

for var in vars:
    title='off'
    ylabel='{} diff. ({})'.format(ds1[var].attrs['name'], ds1[var].attrs['units'])
    map_diff_ave(ds1_mean, ds2_mean, var, 
                 vmin=-vmaxs[vars.index(var)], vmax=vmaxs[vars.index(var)],
                title=title, 
                clabel=ylabel,
                cmap=cmaps[vars.index(var)],
                )
    if savefig:
        plt.savefig('figures/chap4/future/diffmap_JJA_{}_presfut.png'.format(var), bbox_inches='tight', dpi=300)

### Relative diff map (annual)

In [ ]:
savefig=True
vars=['precip', 'evap', 'fluxsens', 'SWdnSFC', 'LWdnSFC', 'q2m', 'rh2m', 'psol']
# vars=['precip']
vmaxs=[30, 20, 45, 10, 5, 10, 8, 0.2]
cmaps=[emb_neutral, emb_neutral, emb, emb, emb, emb_neutral, emb_neutral, emb]

ds1= ip_lmdz_fut_noirr
ds2= ip_lmdz_pres_noirr

ds1_mean = ds1.mean(dim='time')
ds2_mean = ds2.mean(dim='time')

for var in vars:
    title='off'
    ylabel='{} rel. diff. (%)'.format(ds1[var].attrs['name'])
    map_rel_diff_ave(ds1_mean, ds2_mean, var, 
                 vmin=-vmaxs[vars.index(var)], vmax=vmaxs[vars.index(var)],
                title=title, 
                clabel=ylabel,
                cmap=cmaps[vars.index(var)],
                )
    if savefig:
        plt.savefig('figures/chap4/future/reldiffmap_{}_presfut.png'.format(var), bbox_inches='tight', dpi=300)

In [ ]:
savefig=True
vars=['SWdnSFC', 'LWdnSFC', 'netrad', 'cldt', 'cldl', 'cldm', 'cldh']
vmaxs=[10, 10, 10, 15, 25, 20, 10]
cmap=emb

ds1= ip_lmdz_fut_noirr
ds2= ip_lmdz_pres_noirr

ds1_mean = ds1.mean(dim='time')
ds2_mean = ds2.mean(dim='time')

for var in vars:
    title='off'
    ylabel='{} rel. diff. (%)'.format(ds1[var].attrs['name'])
    map_rel_diff_ave(ds1_mean, ds2_mean, var, 
                 vmin=-vmaxs[vars.index(var)], vmax=vmaxs[vars.index(var)],
                title=title, 
                clabel=ylabel,
                cmap=cmap,
                )
    if savefig:
        plt.savefig('figures/chap4/future/reldiffmap_{}_presfut.png'.format(var), bbox_inches='tight', dpi=300)

## Future only, irr vs noirr

### Irr maps and SC

In [ ]:
savefig=True
ds=ip_orc_fut_irr
var = 'irrig_frac'
map_ave(ds,var, cmap=reds,
        vmin=0, vmax=50,
        title='off', clabel='Grid cell irrigation fraction (%)')
if savefig:
    plt.savefig('figures/chap4/future/map_{}_fut.png'.format(var), bbox_inches='tight', dpi=300)

In [ ]:
savefig=False
ds=ip_orc_fut_irr
var = 'root_deficit'
map_ave(ds,var, cmap=reds,
        # vmin=0, vmax=50,
        title='off', 
        # clabel='Grid cell irrigation fraction (%)'
        )
if savefig:
    plt.savefig('figures/chap4/future/map_{}_fut.png'.format(var), bbox_inches='tight', dpi=300)

In [ ]:
savefig=True
ds=ip_orc_fut_irr
var = 'netirrig'
map_ave(ds,var, cmap=blues,
        vmin=0, vmax=6,
        title='off', clabel='Irrigation requirement (mm d⁻¹)')
if savefig:
    plt.savefig('figures/chap4/future/map_{}_fut.png'.format(var), bbox_inches='tight', dpi=300)

time_series_ave([ds], var, year_min=2050, year_max=2062)

seasonal_cycle_ave([ds], var, year_min=2050, year_max=2062,
                   title='off',
                   ylabel='Irrigation requirement (mm d⁻¹)'
                        )
if savefig:
    plt.savefig('figures/chap4/future/SC_{}_fut.png'.format(var), bbox_inches='tight', dpi=300)

In [ ]:
savefig = True
ds=ip_orc_fut_irr
var = 'irrigation'
map_ave(ds,var, cmap=wet,
        vmin=0, vmax=0.75,
        title='off', clabel='Irrigation (mm d⁻¹)')
if savefig:
    plt.savefig('figures/chap4/future/map_{}_fut.png'.format(var), bbox_inches='tight', dpi=300)

time_series_ave([ds], var, year_min=2050, year_max=2062)

seasonal_cycle_ave([ds], var, year_min=2050, year_max=2062, 
                   title='off',
                   ylabel='Irrigation (mm d⁻¹)'
)
if savefig:
    plt.savefig('figures/chap4/future/SC_{}_fut.png'.format(var), bbox_inches='tight', dpi=300)

### Annual diff maps

In [ ]:
savefig=True
vars=['precip', 'evap', 't2m', 'fluxsens', 'q2m', 's_pblh', 's_lcl', 'rh2m']
# vars=['precip']

vmaxs=[0.25, 0.75, 0.5, 15, 1e-3, 100, 200, 4]
cmaps=[emb_neutral, emb_neutral, emb, emb, emb_neutral, emb, emb, emb_neutral]

ds1= ip_lmdz_fut_irr
ds2= ip_lmdz_fut_noirr

for var in vars:
    title='off'
    ylabel='{} diff. ({})'.format(ds1[var].attrs['name'], ds1[var].attrs['units'])
    map_diff_ave(ds1, ds2, var, 
                 vmin=-vmaxs[vars.index(var)], vmax=vmaxs[vars.index(var)],
                title=title, 
                clabel=ylabel,
                cmap=cmaps[vars.index(var)],
                # sig=True,
                )
    if savefig:
        plt.savefig('figures/chap4/future/diffmap_{}_futirr.png'.format(var), bbox_inches='tight', dpi=300)

In [ ]:
savefig=False
vars=['rain', 'humtot', 'runoff', 'drainage', 'slowr', 'fastr', 'streamr']
# vars=['twbr']
# vars=['precip']
vmaxs=[100, 100, 100]

cmap = emb_neutral


for var in vars:
    # map_ave(ip_orc_pres_noirr, var)
    # map_ave(ip_orc_fut_noirr, var)
    # map_ave(ip_orc_fut_irr, var)

    if savefig:
        plt.savefig('figures/chap4/future/reldiffmap_{}_futirr.png'.format(var), bbox_inches='tight', dpi=300)

In [ ]:
savefig=False
vars=['rain', 'humtot', 'runoff', 'drainage', 'slowr', 'fastr', 'streamr']
# vars=['twbr']
# vars=['precip']
vmaxs=[100, 100, 100]

cmap = emb_neutral

for var in vars:
    map_diff_ave(ip_orc_fut_irr, ip_orc_fut_noirr, var,
                 cmap=cmap
            )
    map_rel_diff_ave(ip_orc_fut_irr, ip_orc_fut_noirr, var,
                 cmap=cmap,
                 vmin=-100,
                 vmax=100
            )

    if savefig:
        plt.savefig('figures/chap4/future/reldiffmap_{}_futirr.png'.format(var), bbox_inches='tight', dpi=300)

### Summer diff maps

In [ ]:
savefig=False
vars=['precip', 'evap', 't2m', 'fluxsens', 'q2m', 's_pblh', 's_lcl', 'rh2m']
# vars=['precip']

vmaxs=[0.5, 1.8, 1.25, 40, 1.8e-3, 250, 500, 10]
cmaps=[emb_neutral, emb_neutral, emb, emb, emb_neutral, emb, emb, emb_neutral]

ds1= ip_lmdz_fut_irr
ds2= ip_lmdz_fut_noirr

months=[6,7,8]  # JJA
# months=[0,10,11]  # MAM
ds1 = ds1.sel(time=ds1['time.month'].isin(months))
ds2 = ds2.sel(time=ds2['time.month'].isin(months))

for var in vars:
    title='off'
    ylabel='{} diff. ({})'.format(ds1[var].attrs['name'], ds1[var].attrs['units'])
    map_diff_ave(ds1, ds2, var, 
                 vmin=-vmaxs[vars.index(var)], vmax=vmaxs[vars.index(var)],
                title=title, 
                clabel=ylabel,
                cmap=cmaps[vars.index(var)],
                # sig=True,
                )
    if savefig:
        plt.savefig('figures/chap4/future/diffmap_JJA_{}_futirr.png'.format(var), bbox_inches='tight', dpi=300)

### Annual rel diff maps

In [ ]:
savefig=True
vars=['precip', 'evap', 'fluxsens', 's_pblh', 's_lcl', 'q2m', 'rh2m', 'psol']
# vars=['precip']

vmaxs=[10, 75, 25, 10, 15, 10, 10, 0.02]
cmaps=[emb_neutral, emb_neutral, emb, emb, emb,emb_neutral, emb_neutral, emb]

ds1= ip_lmdz_fut_irr
ds2= ip_lmdz_fut_noirr

for var in vars:
    title='off'
    ylabel='{} rel. diff. (%)'.format(ds1[var].attrs['name'])
    map_rel_diff_ave(ds1, ds2, var, 
                 vmin=-vmaxs[vars.index(var)], vmax=vmaxs[vars.index(var)],
                title=title, 
                clabel=ylabel,
                cmap=cmaps[vars.index(var)],
                )
    if savefig:
        plt.savefig('figures/chap4/future/reldiffmap_{}_futirr.png'.format(var), bbox_inches='tight', dpi=300)

## Aridity index

### Maps

In [ ]:
def penman_monteith_pet(ds_lmdz,ds_orc) -> xr.DataArray:
    """
    Computes Potential Evapotranspiration (PET) using the Penman-Monteith equation,
    incorporating the bulk drag coefficient (cdrag) for aerodynamic resistance.
    All constants and intermediate calculations are performed inline.

    Args:
        ds: xarray.Dataset containing the necessary model variables.
            Requires: netrad (W/m^2), Qg (W/m^2), tair (K), qair (kg/kg),
                      Wind (m/s), psol (Pa), and cdrag (dimensionless, bulk drag coefficient).

    Returns:
        xr.DataArray: Potential Evapotranspiration in mm/day.
    """
    # --- 1. Physical Constants and Standard Values ---
    # Specific heat of air at constant pressure (J kg^-1 K^-1)
    CP_AIR = 1004.0
    # Ratio of molecular weight of water vapor/dry air (dimensionless)
    EPSILON = 0.622
    # Gas constant for dry air (J kg^-1 K^-1)
    R_D = 287.0
    # Density of water (kg m^-3)
    RHO_W = 1000.0

    # --- 2. Extract Variables (Ensure correct units) ---
    Rn = ds_orc['netrad']
    G = ds_orc['Qg']
    Ta = ds_orc['tair']   # Air Temperature (K)
    qa = ds_orc['qair']   # Specific Humidity (kg/kg)
    u = ds_orc['Wind']    # Wind Speed (m/s)
    P = ds_lmdz['psol'] * 100  # Surface Pressure (Pa)
    Cdrag = ds_orc['cdrag'] # Dynamic Bulk Drag Coefficient (dimensionless)

    # --- 3. Intermediate Calculations ---
    # Convert Air Temperature to Celsius
    tair_c = Ta - 273.15

    # 3.1 Latent Heat of Vaporization (Lv) in J kg^-1 (Temperature dependent)
    # Lv = 2.501 x 10^6 - 2361 * T_c
    Lv = (2.501e6 - 2361.0 * tair_c)

    # 3.2 Saturation Vapour Pressure (es) in Pa (Tetens formula)
    # es = 0.6108 * exp((17.27 * T_c) / (T_c + 237.3)) * 1000 (kPa to Pa)
    es = (0.6108 * np.exp((17.27 * tair_c) / (tair_c + 237.3))) * 1000.0

    # 3.3 Slope of the Saturation Vapour Pressure Curve (Delta) in Pa K^-1
    # Delta = (4098 * es) / (T_c + 237.3)^2
    Delta = (4098.0 * es) / ((tair_c + 237.3) ** 2)

    # 3.4 Psychrometric Constant (gamma) in Pa K^-1
    # gamma = (CP_AIR * P) / (EPSILON * L_v)
    gamma = (CP_AIR * P) / (EPSILON * Lv)

    # 3.5 Air Density (rho_a) in kg m^-3 (assuming dry air for simplicity)
    # rho_a = P / (R_D * T_a)
    rho_a = P / (R_D * Ta)

    # 3.6 Aerodynamic Resistance (ra) in s m^-1
    # ra = 1 / (Cdrag * u)
    # This uses the bulk transfer coefficient (Cdrag) and wind speed (u)
    ra = 1.0 / (Cdrag * u)

    # 3.7 Actual Vapour Pressure (ea) in Pa (Simplified from specific humidity)
    # ea ≈ P * qa / EPSILON
    ea = P * qa / EPSILON
    VPD = es - ea # Vapour Pressure Deficit (Pa)

    # --- 4. Penman-Monteith Equation for Latent Heat Flux (LE) ---

    # Numerator - Energy Term: Delta * (Rn - G)
    term_energy = Delta * (Rn - G)

    # Numerator - Aerodynamic Term: rho * cp * VPD / ra
    term_aero = rho_a * CP_AIR * VPD / ra

    # Denominator: Delta + gamma (1 + rc/ra)
    # For PET, surface resistance rc is set to zero (rc = 0), so Denom = Delta + gamma
    denominator = Delta + gamma

    # LE (W/m^2)
    LE = (term_energy + term_aero) / denominator
    LE.name = 'LE_PET'
    LE.attrs['units'] = 'W/m^2'

    # --- 5. Convert LE (W/m^2) to PET (mm/day) ---

    # PET (mm/day) = (LE * seconds_per_day) / (rho_w * L_v)
    pet_mm_day = (LE * 86400.0) / (RHO_W * Lv) * 1000 
    pet_mm_day.name = 'PET_mm_day'
    pet_mm_day.attrs['units'] = 'mm/day'
    pet_mm_day.attrs['long_name'] = 'Potential Evapotranspiration (Penman-Monteith)'

    # Attach the latent heat flux result to the final DataArray for completeness
    pet_mm_day = pet_mm_day.assign_coords(LE_PET=LE)

    ds_orc['PET_PM'] = pet_mm_day
    return ds_orc

In [ ]:
ip_orc_pres_noirr = penman_monteith_pet(ip_lmdz_pres_noirr, ip_orc_pres_noirr)
ip_orc_fut_irr = penman_monteith_pet(ip_lmdz_fut_irr, ip_orc_fut_irr)
ip_orc_fut_noirr = penman_monteith_pet(ip_lmdz_fut_noirr, ip_orc_fut_noirr)

In [ ]:
def compute_annual_aridity_index(ds_lmdz, ds_orc):
    """
    Compute the annual aridity index (P/PET) for the given datasets.
    """
    # Compute annual sums and means
    annual_precip = ds_lmdz['precip'].mean(dim='time')
    annual_pet = ds_orc['PET_PM'].mean(dim='time')

    # Compute aridity index
    aridity_index = (annual_precip / annual_pet)
    aridity_index.attrs['name'] = 'Annual Aridity Index (P/PET)'
    aridity_index.attrs['units'] = '-'

    return aridity_index

In [ ]:
ai_pres_noirr = compute_annual_aridity_index(ip_lmdz_pres_noirr, ip_orc_pres_noirr)
ai_fut_noirr = compute_annual_aridity_index(ip_lmdz_fut_noirr, ip_orc_fut_noirr)
ai_fut_irr = compute_annual_aridity_index(ip_lmdz_fut_irr, ip_orc_fut_irr)

In [ ]:
savefig_AI=True

In [ ]:
map_plotvar(ai_pres_noirr, cmap='Spectral', vmin=0, vmax=1, title='', clabel='Aridity Index (P/PET)')
if savefig_AI:
    plt.savefig('figures/chap4/future/map_AI_pres_noirr.png', bbox_inches='tight', dpi=300)

In [ ]:
map_plotvar(ai_fut_noirr, cmap='Spectral', vmin=0, vmax=1, title='', clabel='Aridity Index (P/PET)')
if savefig_AI:
    plt.savefig('figures/chap4/future/map_AI_fut_noirr.png', bbox_inches='tight', dpi=300)

In [ ]:
map_plotvar(ai_fut_irr, cmap='Spectral', vmin=0, vmax=1, title='', clabel='Aridity Index (P/PET)')
if savefig_AI:
    plt.savefig('figures/chap4/future/map_AI_fut_irr.png', bbox_inches='tight', dpi=300)

In [ ]:
map_aridity_index(ip_lmdz_pres_noirr, ip_orc_pres_noirr)
if savefig_AI:
    plt.savefig("figures/chap4/future/aridity_index_pres_noirr.png", dpi=300)

In [ ]:
map_aridity_index(ip_lmdz_fut_noirr, ip_orc_fut_noirr)
if savefig_AI:
    plt.savefig("figures/chap4/future/aridity_index_fut_noirr.png", dpi=300)

In [ ]:
map_aridity_index(ip_lmdz_fut_irr, ip_orc_fut_irr)
if savefig_AI:
    plt.savefig("figures/chap4/future/aridity_index_fut_irr.png", dpi=300)

In [ ]:
savefig=True
ds1_lmdz = ip_lmdz_pres_noirr
ds1_orc = ip_orc_pres_noirr
ds2_lmdz = ip_lmdz_fut_noirr
ds2_orc = ip_orc_fut_noirr

# map_aridity_index_change(ds1_lmdz, ds1_orc, ds2_lmdz, ds2_orc, clabel="Change in Aridity Index (no irrigation)", left_labels=True)
abs, rel = compute_aridity_index_change(ds1_lmdz, ds1_orc, ds2_lmdz, ds2_orc)
map_plotvar(abs, cmap=emb_neutral, vmin=-0.3, vmax=0.3, clabel="Aridity Index difference")
if savefig:
    plt.savefig("figures/chap4/future/diffmap_aridity_index_presfut.png", dpi=300)
map_plotvar(rel, cmap=emb_neutral, vmin=-40, vmax=40, clabel="Aridity Index relative difference (%)")
if savefig:
    plt.savefig("figures/chap4/future/reldiffmap_aridity_index_presfut.png", dpi=300)

In [ ]:
savefig=True
ds1_lmdz = ip_lmdz_fut_noirr
ds1_orc = ip_orc_fut_noirr
ds2_lmdz = ip_lmdz_fut_irr
ds2_orc = ip_orc_fut_irr

# map_aridity_index_change(ds1_lmdz, ds1_orc, ds2_lmdz, ds2_orc, clabel="Change in Aridity Index (no irrigation)", left_labels=True)
abs, rel = compute_aridity_index_change(ds1_lmdz, ds1_orc, ds2_lmdz, ds2_orc)
map_plotvar(abs, cmap=emb_neutral, 
            vmin=-0.1, vmax=0.1, 
            clabel="Aridity Index difference")
if savefig:
    plt.savefig("figures/chap4/future/diffmap_aridity_index_futirr.png", dpi=300)

map_plotvar(rel, cmap=emb_neutral, 
            vmin=-20, vmax=20, 
            clabel="Aridity Index relative difference (%)")
if savefig:
    plt.savefig("figures/chap4/future/reldiffmap_aridity_index_futirr.png", dpi=300)

### Numbers and distribution

In [ ]:
def ai_distribution(ds_lmdz, ds_orc):
    precip = ds_lmdz['precip'].mean('time')
    evapot = ds_orc['PET_PM'].mean('time') 
    ai = (precip / evapot)   

    bounds = [0, 0.05, 0.2, 0.5, 0.65, np.inf]
    colors = ['red', 'orange', 'yellow', 'lightskyblue', 'blue']
    labels = ["Hyperarid", "Arid", "Semiarid", "Dry subhumid", "Humid"]

    # Digitize AI values into class indices (0..4)
    class_idx = np.digitize(ai.values.flatten(), bounds) - 1

    # Count grid cells in each class
    counts = [np.sum(class_idx == i) for i in range(len(labels))]
    total = np.sum(counts)
    fractions = [count / total * 100 for count in counts]

    # Print results
    for label, count, fraction in zip(labels, counts, fractions):
        print(f"%{label}: {count} grid cells ({fraction})")
    print(f"%Total: {total} grid cells")

    # Bar chart
    plt.figure(figsize=(4,4))
    #bar with vmax height of 100%
    plt.bar(labels, fractions, color=colors, width=0.5)
    plt.ylim(0, 67)
    plt.ylabel("Fraction of domain (%)")
    # plt.title("Aridity Index Distribution")
    plt.xticks(rotation=30, ha='right')
    plt.tight_layout()

    return counts


In [ ]:
savefig=True

ds_lmdz=ip_lmdz_pres_noirr
ds_orc = ip_orc_pres_noirr
print('% present_noirr')
ai_distribution(ds_lmdz, ds_orc)
if savefig:
    plt.savefig("figures/chap4/future/aridity_index_distribution_pres_noirr.png", dpi=300)

ds_lmdz=ip_lmdz_fut_noirr
ds_orc = ip_orc_fut_noirr
print('% future_noirr')
ai_distribution(ds_lmdz, ds_orc)
if savefig:
    plt.savefig("figures/chap4/future/aridity_index_distribution_fut_noirr.png", dpi=300)

ds_lmdz=ip_lmdz_fut_irr
ds_orc = ip_orc_fut_irr
print('% future_irr')
ai_distribution(ds_lmdz, ds_orc)
if savefig:
    plt.savefig("figures/chap4/future/aridity_index_distribution_fut_irr.png", dpi=300)

In [ ]:
stop

## Appendix

### Seasonnal diff maps for precip, ET, t2m

todo

In [ ]:
savefig=True
vars=['precip', 'evap', 't2m']
# vars=['precip']
vmaxs=[2, 0.5, 3]
cmaps=[emb_neutral, emb_neutral, emb]

ds1= ip_lmdz_fut_noirr
ds2= ip_lmdz_pres_noirr
#NB : pas straightforward car pas la même période, besoin de faire la moyenne avant mais en gardant les saisons

### Mean SC for 3 sims (present no_irr, future irr & no_irr)

todo: ideally add present irr but not available yet...

In [ ]:
savefig=False
include_irr=True
vars=['precip', 'evap', 't2m', 'fluxsens', 'q2m', 's_pblh', 's_lcl', 'rh2m', 'LWdnSFC', 'SWdnSFC', 'netrad', 'cldt', 'cldl', 'cldm', 'cldh']
# vars=['precip']
vmins=[0, 0, 0, 0, ]
vmaxs=[4, 4, 30, 0.012]

ds1= ip_lmdz_pres_noirr
ds2= ip_lmdz_fut_noirr
ds3= ip_lmdz_fut_irr

year_min, year_max = 2010, 2062

ds_list = [ds1, ds2]
if include_irr:
    ds_list.append(ds3)
    
for var in vars:
    title='off'
    ylabel='{} ({})'.format(ds1[var].attrs['name'], ds1[var].attrs['units'])
    # time_series_ave(ds_list, var,
    #                 title=title,
    #                 ylabel=ylabel,
    #                 ds_colors=True,
    #                 year_min=year_min, year_max=year_max)
    
    seasonal_cycle_ave(ds_list, var, 
                    title=title,
                    ylabel=ylabel,
                    ds_colors=True,
                    year_min=year_min, year_max=year_max,
                    # vmin=vmins[vars.index(var)],
                    # vmax=vmaxs[vars.index(var)]
                    )   
    if savefig:
        plt.savefig('figures/chap4/future/SC_{}_presfutirr.png'.format(var), bbox_inches='tight', dpi=300)